In [1]:
import pandas as pd
import numpy as np

pm=open('ppm_98_TMRstart.fas','r')
npm=open('npm_98_TMRstart.fas','r')

fas = [[]]

while True:
    reading = pm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_pm=pd.DataFrame(fas,columns=columns)

fas = [[]]

while True:
    reading = npm.readline()
    reading = reading.rstrip()

    if not reading.startswith('>'):
        if reading == '':
            break
        fas.append(list(reading))

fas.remove([])
columns=list(range(-25,26))
df_npm=pd.DataFrame(fas,columns=columns)

In [2]:
position = -25

df_pm_freq=df_pm[position].value_counts(sort=True)
df_pm_nor=df_pm[position].value_counts(sort=True,normalize=True)

df_npm_freq=df_npm[position].value_counts(sort=True)
df_npm_nor=df_npm[position].value_counts(sort=True,normalize=True)

printout=pd.concat([df_pm_freq,df_pm_nor,df_npm_freq,df_npm_nor],axis='columns')

printout

,-25,-25,-25,-25
X,32.0,0.410256,395,0.701599
D,5.0,0.064103,7,0.012433
T,5.0,0.064103,7,0.012433
K,4.0,0.051282,8,0.014210
Q,4.0,0.051282,8,0.014210
E,4.0,0.051282,15,0.026643
P,4.0,0.051282,16,0.028419
I,3.0,0.038462,3,0.005329
L,2.0,0.025641,23,0.040853
A,2.0,0.025641,14,0.024867
